# CloudWalk Operations Intelligence - Exploratory Data Analysis

**Author**: Gabriel Milhardo  
**Date**: January 2025  
**Objective**: Understand the transaction data structure, quality, and key patterns to build an AI-powered analytics agent.

---

## Table of Contents
1. [Data Loading & Initial Inspection](#1-data-loading--initial-inspection)
2. [Data Quality Check](#2-data-quality-check)
3. [Univariate Analysis](#3-univariate-analysis)
4. [Time Series Analysis](#4-time-series-analysis)
5. [Segmentation Analysis](#5-segmentation-analysis)
6. [Key Insights Summary](#6-key-insights-summary)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('Libraries imported successfully!')

Libraries imported successfully!


---
## 1. Data Loading & Initial Inspection

Let's load the CloudWalk transaction data and understand its basic structure.

In [2]:
# Load data
df = pd.read_csv('../Operations_analyst_data.csv')

print('='*60)
print('DATASET OVERVIEW')
print('='*60)
print(f'\nShape: {df.shape[0]:,} rows x {df.shape[1]} columns')
print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

DATASET OVERVIEW

Shape: 37,787 rows x 10 columns
Memory usage: 13.25 MB


In [3]:
# Display first rows
print('\nFirst 5 rows:')
df.head()


First 5 rows:


,day,entity,product,price_tier,anticipation_method,payment_method,installments,amount_transacted,quantity_transactions,quantity_of_merchants
0,2025-01-01,PJ,pix,intermediary,Pix,uninformed,1,"546,030.68",13361,675
1,2025-01-01,PJ,pos,intermediary,D1Anticipation,credit,1,"574,568.13",9104,694
2,2025-01-01,PJ,tap,normal,D0/Nitro,credit,1,"342,178.87",2655,1607
3,2025-01-01,PJ,tap,domination,D1Anticipation,debit,1,"1,183.21",63,9
4,2025-01-01,PJ,tap,aggressive,D0/Nitro,credit,1,"10,158.11",30,20


In [4]:
# Display last rows
print('\nLast 5 rows:')
df.tail()


Last 5 rows:


,day,entity,product,price_tier,anticipation_method,payment_method,installments,amount_transacted,quantity_transactions,quantity_of_merchants
37782,2025-03-31,PF,tap,domination,D1Anticipation,credit,12,"58,998.78",9,7
37783,2025-03-31,PF,tap,aggressive,D0/Nitro,credit,12,"3,078.44",1,1
37784,2025-03-31,PJ,link,aggressive,D0/Nitro,credit,12,"9,125.96",7,4
37785,2025-03-31,PJ,pos,domination,D1Anticipation,credit,12,"2,258,321.18",552,286
37786,2025-03-31,PJ,pos,intermediary,D0/Nitro,credit,12,"35,000.00",1,1


In [5]:
# Data types and info
print('\nColumn Data Types:')
print(df.dtypes)
print('\n' + '='*60)
df.info()


Column Data Types:
day                       object
entity                    object
product                   object
price_tier                object
anticipation_method       object
payment_method            object
installments               int64
amount_transacted        float64
quantity_transactions      int64
quantity_of_merchants      int64
dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37787 entries, 0 to 37786
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   day                    37787 non-null  object 
 1   entity                 37787 non-null  object 
 2   product                37787 non-null  object 
 3   price_tier             37787 non-null  object 
 4   anticipation_method    37787 non-null  object 
 5   payment_method         37787 non-null  object 
 6   installments           37787 non-null  int64  
 7   amount_transacted      37787 non-null  float64
 8   q

In [6]:
# Convert 'day' to datetime
df['day'] = pd.to_datetime(df['day'])

print('Date column converted to datetime!')
print(f'\nDate Range: {df["day"].min()} to {df["day"].max()}')
print(f'Total Days: {df["day"].nunique()} unique days')

Date column converted to datetime!

Date Range: 2025-01-01 00:00:00 to 2025-03-31 00:00:00
Total Days: 90 unique days


---
## 2. Data Quality Check

Let's verify data quality: missing values, duplicates, and anomalies.

In [7]:
# Missing values analysis
print('='*60)
print('MISSING VALUES ANALYSIS')
print('='*60)

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})

print(missing_df)
print(f'\nTotal missing values: {missing.sum()}')

MISSING VALUES ANALYSIS
                       Missing Count  Missing %
day                                0       0.00
entity                             0       0.00
product                            0       0.00
price_tier                         0       0.00
anticipation_method                0       0.00
payment_method                     0       0.00
installments                       0       0.00
amount_transacted                  0       0.00
quantity_transactions              0       0.00
quantity_of_merchants              0       0.00

Total missing values: 0


In [8]:
# Duplicate check
print('\n' + '='*60)
print('DUPLICATE CHECK')
print('='*60)

duplicates = df.duplicated().sum()
print(f'\nTotal duplicate rows: {duplicates}')
print(f'Duplicate percentage: {duplicates/len(df)*100:.2f}%')


DUPLICATE CHECK

Total duplicate rows: 0
Duplicate percentage: 0.00%


In [9]:
# Date continuity check
print('\n' + '='*60)
print('DATE CONTINUITY CHECK')
print('='*60)

date_range = pd.date_range(start=df['day'].min(), end=df['day'].max())
missing_dates = set(date_range) - set(df['day'].unique())

print(f'\nExpected days in range: {len(date_range)}')
print(f'Actual unique days: {df["day"].nunique()}')
print(f'Missing dates: {len(missing_dates)}')

if missing_dates:
    print('\nMissing dates list:')
    for d in sorted(missing_dates)[:10]:
        print(f'  - {d.strftime("%Y-%m-%d")}')
    if len(missing_dates) > 10:
        print(f'  ... and {len(missing_dates) - 10} more')


DATE CONTINUITY CHECK

Expected days in range: 90
Actual unique days: 90
Missing dates: 0


In [10]:
# Numerical columns sanity check
print('\n' + '='*60)
print('NUMERICAL SANITY CHECK')
print('='*60)

numerical_cols = ['amount_transacted', 'quantity_transactions', 'quantity_of_merchants', 'installments']

for col in numerical_cols:
    negative = (df[col] < 0).sum()
    zeros = (df[col] == 0).sum()
    print(f'\n{col}:')
    print(f'  Negative values: {negative}')
    print(f'  Zero values: {zeros}')
    print(f'  Min: {df[col].min():,.2f}, Max: {df[col].max():,.2f}')


NUMERICAL SANITY CHECK

amount_transacted:
  Negative values: 0
  Zero values: 0
  Min: 1.00, Max: 28,088,890.22

quantity_transactions:
  Negative values: 0
  Zero values: 0
  Min: 1.00, Max: 541,644.00

quantity_of_merchants:
  Negative values: 0
  Zero values: 0
  Min: 1.00, Max: 96,909.00

installments:
  Negative values: 0
  Zero values: 0
  Min: 1.00, Max: 12.00


---
## 3. Univariate Analysis

### 3.1 Categorical Variables

In [11]:
# Categorical columns overview
categorical_cols = ['entity', 'product', 'price_tier', 'anticipation_method', 'payment_method']

print('='*60)
print('CATEGORICAL VARIABLES - UNIQUE VALUES')
print('='*60)

for col in categorical_cols:
    print(f'\n{col.upper()} ({df[col].nunique()} unique):')
    value_counts = df[col].value_counts()
    for val, count in value_counts.items():
        pct = count / len(df) * 100
        print(f'  - {val}: {count:,} ({pct:.1f}%)')

CATEGORICAL VARIABLES - UNIQUE VALUES

ENTITY (2 unique):
  - PJ: 24,264 (64.2%)
  - PF: 13,523 (35.8%)

PRODUCT (5 unique):
  - tap: 13,263 (35.1%)
  - link: 11,958 (31.6%)
  - pos: 11,429 (30.2%)
  - pix: 709 (1.9%)
  - bank_slip: 428 (1.1%)

PRICE_TIER (4 unique):
  - normal: 13,563 (35.9%)
  - aggressive: 9,519 (25.2%)
  - intermediary: 7,985 (21.1%)
  - domination: 6,720 (17.8%)

ANTICIPATION_METHOD (4 unique):
  - D1Anticipation: 20,763 (54.9%)
  - D0/Nitro: 15,887 (42.0%)
  - Pix: 709 (1.9%)
  - Bank Slip: 428 (1.1%)

PAYMENT_METHOD (3 unique):
  - credit: 34,119 (90.3%)
  - debit: 2,531 (6.7%)
  - uninformed: 1,137 (3.0%)


In [12]:
# Installments distribution
print('\n' + '='*60)
print('INSTALLMENTS DISTRIBUTION')
print('='*60)

print(df['installments'].value_counts().sort_index())


INSTALLMENTS DISTRIBUTION
installments
1     7501
2     3511
3     3402
4     3154
5     3062
6     2992
7     2282
8     2447
9     1925
10    3149
11    1472
12    2890
Name: count, dtype: int64


### 3.2 Numerical Variables Statistics

In [13]:
# Descriptive statistics
print('='*60)
print('NUMERICAL VARIABLES - DESCRIPTIVE STATISTICS')
print('='*60)

df[numerical_cols].describe()

NUMERICAL VARIABLES - DESCRIPTIVE STATISTICS


,amount_transacted,quantity_transactions,quantity_of_merchants,installments
count,"37,787.00","37,787.00","37,787.00","37,787.00"
mean,"514,532.37","3,876.89","1,259.95",5.45
std,"1,573,947.11","26,790.76","6,663.14",3.63
min,1.00,1.00,1.00,1.00
25%,"4,842.70",4.00,3.00,2.00
50%,"35,846.06",28.00,22.00,5.00
75%,"330,312.30",361.00,268.00,8.00
max,"28,088,890.22","541,644.00","96,909.00",12.00


In [14]:
# Key metrics calculation
print('\n' + '='*60)
print('KEY METRICS (TOTAL PERIOD)')
print('='*60)

total_tpv = df['amount_transacted'].sum()
total_transactions = df['quantity_transactions'].sum()
avg_ticket = total_tpv / total_transactions

print(f'\nTotal TPV (Transaction Volume): R$ {total_tpv:,.2f}')
print(f'Total Transactions: {total_transactions:,.0f}')
print(f'Average Ticket: R$ {avg_ticket:,.2f}')
print(f'\nTPV in Billions: R$ {total_tpv/1e9:.2f}B')


KEY METRICS (TOTAL PERIOD)

Total TPV (Transaction Volume): R$ 19,442,634,813.40
Total Transactions: 146,496,103
Average Ticket: R$ 132.72

TPV in Billions: R$ 19.44B


---
## 4. Time Series Analysis

### 4.1 Daily TPV Trend

In [15]:
# Aggregate by day
daily_metrics = df.groupby('day').agg({
    'amount_transacted': 'sum',
    'quantity_transactions': 'sum',
    'quantity_of_merchants': 'sum'
}).reset_index()

daily_metrics.columns = ['day', 'tpv', 'transactions', 'merchants']
daily_metrics['avg_ticket'] = daily_metrics['tpv'] / daily_metrics['transactions']

print('Daily metrics computed!')
daily_metrics.head()

Daily metrics computed!


,day,tpv,transactions,merchants,avg_ticket
0,2025-01-01,"51,990,755.03",762039,142624,68.23
1,2025-01-02,"156,414,349.55",1173190,363468,133.32
2,2025-01-03,"168,032,949.24",1377057,435746,122.02
3,2025-01-04,"138,069,838.58",1461058,417864,94.50
4,2025-01-05,"91,291,563.25",1007278,238751,90.63


In [16]:
# Daily TPV line chart
fig = px.line(
    daily_metrics, 
    x='day', 
    y='tpv',
    title='Daily TPV (Total Payment Volume)',
    labels={'day': 'Date', 'tpv': 'TPV (R$)'}
)

fig.update_traces(line_color='#2E86AB')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='TPV (R$)',
    yaxis_tickformat=',.0f',
    hovermode='x unified'
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Daily transactions trend
fig = px.line(
    daily_metrics, 
    x='day', 
    y='transactions',
    title='Daily Transaction Count',
    labels={'day': 'Date', 'transactions': 'Transactions'}
)

fig.update_traces(line_color='#A23B72')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of Transactions',
    yaxis_tickformat=',.0f',
    hovermode='x unified'
)

fig.show()

### 4.2 Day of Week Analysis

In [ ]:
# Add day of week
daily_metrics['day_of_week'] = daily_metrics['day'].dt.day_name()
daily_metrics['day_num'] = daily_metrics['day'].dt.dayofweek

# Aggregate by day of week
dow_metrics = daily_metrics.groupby(['day_of_week', 'day_num']).agg({
    'tpv': 'mean',
    'transactions': 'mean'
}).reset_index().sort_values('day_num')

print('Average TPV by Day of Week:')
dow_metrics[['day_of_week', 'tpv', 'transactions']]

In [ ]:
# Day of week bar chart
fig = px.bar(
    dow_metrics,
    x='day_of_week',
    y='tpv',
    title='Average Daily TPV by Day of Week',
    labels={'day_of_week': 'Day of Week', 'tpv': 'Average TPV (R$)'},
    color='tpv',
    color_continuous_scale='Blues'
)

fig.update_layout(
    xaxis_title='Day of Week',
    yaxis_title='Average TPV (R$)',
    yaxis_tickformat=',.0f',
    showlegend=False
)

fig.show()

---
## 5. Segmentation Analysis

### 5.1 TPV by Product

In [ ]:
# TPV by product
product_metrics = df.groupby('product').agg({
    'amount_transacted': 'sum',
    'quantity_transactions': 'sum',
    'quantity_of_merchants': 'sum'
}).reset_index()

product_metrics.columns = ['product', 'tpv', 'transactions', 'merchants']
product_metrics['avg_ticket'] = product_metrics['tpv'] / product_metrics['transactions']
product_metrics['tpv_pct'] = product_metrics['tpv'] / product_metrics['tpv'].sum() * 100
product_metrics = product_metrics.sort_values('tpv', ascending=False)

print('TPV by Product:')
product_metrics

In [ ]:
# Product TPV bar chart
fig = px.bar(
    product_metrics.sort_values('tpv', ascending=True),
    x='tpv',
    y='product',
    orientation='h',
    title='Total TPV by Product',
    labels={'tpv': 'TPV (R$)', 'product': 'Product'},
    color='tpv',
    color_continuous_scale='Viridis',
    text='tpv_pct'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    xaxis_title='TPV (R$)',
    yaxis_title='Product',
    xaxis_tickformat=',.0f',
    showlegend=False
)

fig.show()

### 5.2 TPV by Entity (PJ vs PF)

In [ ]:
# TPV by entity
entity_metrics = df.groupby('entity').agg({
    'amount_transacted': 'sum',
    'quantity_transactions': 'sum'
}).reset_index()

entity_metrics.columns = ['entity', 'tpv', 'transactions']
entity_metrics['avg_ticket'] = entity_metrics['tpv'] / entity_metrics['transactions']
entity_metrics['tpv_pct'] = entity_metrics['tpv'] / entity_metrics['tpv'].sum() * 100

# Add descriptive labels
entity_metrics['entity_label'] = entity_metrics['entity'].map({
    'PJ': 'Business (PJ)',
    'PF': 'Individual (PF)'
})

print('TPV by Entity:')
entity_metrics

In [ ]:
# Entity pie chart
fig = px.pie(
    entity_metrics,
    values='tpv',
    names='entity_label',
    title='TPV Distribution: Business vs Individual',
    color_discrete_sequence=['#2E86AB', '#A23B72']
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### 5.3 TPV by Payment Method

In [ ]:
# TPV by payment method
payment_metrics = df.groupby('payment_method').agg({
    'amount_transacted': 'sum',
    'quantity_transactions': 'sum'
}).reset_index()

payment_metrics.columns = ['payment_method', 'tpv', 'transactions']
payment_metrics['avg_ticket'] = payment_metrics['tpv'] / payment_metrics['transactions']
payment_metrics['tpv_pct'] = payment_metrics['tpv'] / payment_metrics['tpv'].sum() * 100
payment_metrics = payment_metrics.sort_values('tpv', ascending=False)

print('TPV by Payment Method:')
payment_metrics

In [ ]:
# Payment method bar chart
fig = px.bar(
    payment_metrics,
    x='payment_method',
    y='tpv',
    title='Total TPV by Payment Method',
    labels={'payment_method': 'Payment Method', 'tpv': 'TPV (R$)'},
    color='payment_method',
    color_discrete_sequence=['#2E86AB', '#A23B72', '#F18F01']
)

fig.update_layout(
    xaxis_title='Payment Method',
    yaxis_title='TPV (R$)',
    yaxis_tickformat=',.0f',
    showlegend=False
)

fig.show()

### 5.4 Heatmap: Product x Day of Week

In [ ]:
# Add day of week to main dataframe
df['day_of_week'] = df['day'].dt.day_name()
df['day_num'] = df['day'].dt.dayofweek

# Pivot table for heatmap
heatmap_data = df.groupby(['product', 'day_of_week', 'day_num'])['amount_transacted'].sum().reset_index()
heatmap_pivot = heatmap_data.pivot_table(
    index='product', 
    columns='day_num', 
    values='amount_transacted',
    aggfunc='mean'
)

# Rename columns to day names
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_pivot.columns = day_names

heatmap_pivot

In [ ]:
# Heatmap visualization
fig = px.imshow(
    heatmap_pivot,
    title='Average TPV: Product x Day of Week',
    labels=dict(x='Day of Week', y='Product', color='TPV (R$)'),
    color_continuous_scale='Blues',
    aspect='auto'
)

fig.update_layout(
    xaxis_title='Day of Week',
    yaxis_title='Product'
)

fig.show()

### 5.5 Daily TPV by Product (Time Series)

In [ ]:
# Daily TPV by product
daily_product = df.groupby(['day', 'product'])['amount_transacted'].sum().reset_index()

fig = px.line(
    daily_product,
    x='day',
    y='amount_transacted',
    color='product',
    title='Daily TPV by Product',
    labels={'day': 'Date', 'amount_transacted': 'TPV (R$)', 'product': 'Product'}
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='TPV (R$)',
    yaxis_tickformat=',.0f',
    hovermode='x unified',
    legend_title='Product'
)

fig.show()

---
## 6. Key Insights Summary

### Data Dictionary (Finalized)

In [ ]:
# Create data dictionary
data_dict = {
    'Column': ['day', 'entity', 'product', 'price_tier', 'anticipation_method', 
               'payment_method', 'installments', 'amount_transacted', 
               'quantity_transactions', 'quantity_of_merchants'],
    'Type': ['DATE', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'INTEGER', 'REAL', 'INTEGER', 'INTEGER'],
    'Description': [
        'Transaction date (YYYY-MM-DD)',
        'Client type: PJ (Business) or PF (Individual)',
        'Payment product: pix, pos, tap, link, bank_slip',
        'Pricing tier: normal, intermediary, aggressive, domination',
        'Anticipation method: Pix, D1Anticipation, D0/Nitro, Bank Slip',
        'Payment method: credit, debit, uninformed',
        'Number of installments (1-N)',
        'TPV - Total Payment Volume in BRL',
        'Number of transactions',
        'Number of unique merchants'
    ],
    'Unique Values': [
        str(df['day'].nunique()),
        str(list(df['entity'].unique())),
        str(list(df['product'].unique())),
        str(list(df['price_tier'].unique())),
        str(list(df['anticipation_method'].unique())),
        str(list(df['payment_method'].unique())),
        str(sorted(df['installments'].unique())),
        f'Min: {df["amount_transacted"].min():,.2f}, Max: {df["amount_transacted"].max():,.2f}',
        f'Min: {df["quantity_transactions"].min()}, Max: {df["quantity_transactions"].max()}',
        f'Min: {df["quantity_of_merchants"].min()}, Max: {df["quantity_of_merchants"].max()}'
    ]
}

data_dict_df = pd.DataFrame(data_dict)
print('='*60)
print('DATA DICTIONARY')
print('='*60)
data_dict_df

### Key Findings

In [ ]:
# Summary statistics for key insights
print('='*60)
print('KEY INSIGHTS SUMMARY')
print('='*60)

# 1. Total metrics
print('\n1. OVERALL METRICS:')
print(f'   - Total TPV: R$ {total_tpv:,.2f} ({total_tpv/1e9:.2f}B)')
print(f'   - Total Transactions: {total_transactions:,.0f}')
print(f'   - Average Ticket: R$ {avg_ticket:,.2f}')
print(f'   - Date Range: {df["day"].min().strftime("%Y-%m-%d")} to {df["day"].max().strftime("%Y-%m-%d")}')

# 2. Top product
top_product = product_metrics.iloc[0]
print(f'\n2. TOP PRODUCT by TPV:')
print(f'   - {top_product["product"].upper()}: R$ {top_product["tpv"]:,.2f} ({top_product["tpv_pct"]:.1f}% of total)')

# 3. Entity split
print(f'\n3. ENTITY DISTRIBUTION:')
for _, row in entity_metrics.iterrows():
    print(f'   - {row["entity_label"]}: {row["tpv_pct"]:.1f}% of TPV')

# 4. Payment method
print(f'\n4. PAYMENT METHOD:')
for _, row in payment_metrics.iterrows():
    print(f'   - {row["payment_method"].capitalize()}: {row["tpv_pct"]:.1f}% of TPV, Avg Ticket: R$ {row["avg_ticket"]:,.2f}')

# 5. Busiest day
busiest_day = dow_metrics.loc[dow_metrics['tpv'].idxmax()]
slowest_day = dow_metrics.loc[dow_metrics['tpv'].idxmin()]
print(f'\n5. DAY OF WEEK PATTERNS:')
print(f'   - Busiest: {busiest_day["day_of_week"]} (Avg TPV: R$ {busiest_day["tpv"]:,.2f})')
print(f'   - Slowest: {slowest_day["day_of_week"]} (Avg TPV: R$ {slowest_day["tpv"]:,.2f})')

In [ ]:
# Data quality summary
print('\n' + '='*60)
print('DATA QUALITY SUMMARY')
print('='*60)
print(f'\n- Missing Values: {missing.sum()} (Data is complete)')
print(f'- Duplicates: {duplicates} rows')
print(f'- Date Coverage: {df["day"].nunique()} days')
print(f'- Negative Values: None detected in numerical columns')
print('\n>>> Data is CLEAN and ready for AI Agent implementation!')

---

## Next Steps

Based on this EDA, we will now:

1. **Load data into SQLite** for efficient querying
2. **Build the AI Agent** with LangChain + Groq (Llama 3)
3. **Implement anomaly detection** using Z-score
4. **Create Streamlit interface** for interactive Q&A

---

*End of Exploratory Data Analysis*